#### Auto-fill Questionnaire using Chain of Thought or Few-Shot Examples

This notebook showcases the application of few-shot examples in autofilling questionnaires. It utilizes a json file (`cset_ai_harm_questionnaire.json`) to
provide the LLM with example responses for some use-cases.

By leveraging these few-shot examples, we can enable seamless completion of lengthy questionnaires, minimizing manual effort and improving overall efficiency.


In [4]:
from risk_atlas_nexus.blocks.inference import (
    RITSInferenceEngine,
    WMLInferenceEngine,
    OllamaInferenceEngine,
    VLLMInferenceEngine,
)
from risk_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams,
    OllamaInferenceEngineParams,
    VLLMInferenceEngineParams,
)

from risk_atlas_nexus.data import load_resource
from risk_atlas_nexus.library import RiskAtlasNexus

##### Risk Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/risk-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [5]:
# inference_engine = OllamaInferenceEngine(
#     model_name_or_path="granite3.2:8b",
#     credentials=InferenceEngineCredentials(api_url="http://localhost:11434"),
#     parameters=OllamaInferenceEngineParams(
#         num_predict=1000, temperature=0, repeat_penalty=1, num_ctx=8192
#     ),
# )

# inference_engine = WMLInferenceEngine(
#     model_name_or_path="ibm/granite-20b-code-instruct",
#     credentials={
#         "api_key": "WML_API_KEY",
#         "api_url": "WML_API_URL",
#         "project_id": "WML_PROJECT_ID",
#     },
#     parameters=WMLInferenceEngineParams(
#         max_new_tokens=1000, decoding_method="greedy", repetition_penalty=1
#     ),
# )

# inference_engine = VLLMInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.1-8b-instruct",
#     credentials=InferenceEngineCredentials(
#         api_url="VLLM_API_URL", api_key="VLLM_API_KEY"
#     ),
#     parameters=VLLMInferenceEngineParams(max_tokens=1000, temperature=0.7),
# )

inference_engine = RITSInferenceEngine(
    model_name_or_path="ibm-granite/granite-3.3-8b-instruct",
    credentials={
        "api_key": "cbc683b3a1a7c52d2a73008b785d2811",
        "api_url": "https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com",
    },
    parameters=RITSInferenceEngineParams(max_completion_tokens=1000, temperature=0),
)

[2025-10-20 11:39:04:645] - INFO - RiskAtlasNexus - Created RITS inference engine.


##### Create an instance of RiskAtlasNexus

_Note: (Optional)_ You can specify your own directory in `RiskAtlasNexus(base_dir=<PATH>)` to utilize custom AI ontologies. If left blank, the system will use the provided AI ontologies.


In [6]:
risk_atlas_nexus = RiskAtlasNexus()

[2025-10-20 11:39:04:877] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


#### Defining Examples for Auto-Assist Functionality

The auto-assist feature utilizes few-shot examples defined in the file `risk_atlas_nexus/data/templates/cset_ai_harm_questionnaire.json` to predict the output of the ai harm questionnaire.

**Customization:**

To adapt this auto-assist functionality to custom risk questionnaire, users need to provide their own set of questions, example intents, and corresponding answers in a json file such as in [cset_ai_harm_questionnaire.json](https://github.com/IBM/risk-atlas-nexus/blob/main/src/risk_atlas_nexus/data/templates/cset_ai_harm_questionnaire.json). This will enable the LLM to learn from these few-shot examples and generate responses for unseen queries.

**CoT Template - Zero Shot method**

Each question is accompanied by corresponding examples provided as an empty list.

```shell
  [
      {
          "question": "Did the incident occur in a domain with physical objects? 'Yes' if the AI system(s) is embedded in hardware that can interact, affect, and change with the physical objects (roads, factories, medical facilities, etc.). 'No' if the system cannot. This includes systems that inform, detect, predict, or recommend.",
          "cot_examples": [],
          "response_format": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "enum": [
                        "Yes",
                        "No",
                        "Maybe"
                    ]
                },
                "explanation": {
                    "type": "string"
                }
            },
            "required": [
                "answer",
                "explanation"
            ]
        }
      }
      ...
  ]
```

**CoT Template - Few Shot method**

Each question is associated with a list of examples, each containing intent, answer, and optional explanation.

```shell
  [
      {
          "question": "Did the incident occur in a domain with physical objects? 'Yes' if the AI system(s) is embedded in hardware that can interact, affect, and change with the physical objects (roads, factories, medical facilities, etc.). 'No' if the system cannot. This includes systems that inform, detect, predict, or recommend.",
          "cot_examples": [
            {
                "intent": "Google's Perspective API, which assigns a toxicity score to online text, seems to award higher toxicity scores to content involving non-white, male, Christian, heterosexual phrases.",
                "answer": "No"
            },
            {
                "intent": "An Uber autonomous vehicle (AV) in autonomous mode struck and killed a pedestrian in Tempe, Arizona.",
                "answer": "Yes"
            }
            ...
          ],
          "response_format": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "enum": [
                        "Yes",
                        "No",
                        "Maybe"
                    ]
                },
                "explanation": {
                    "type": "string"
                }
            },
            "required": [
                "answer",
                "explanation"
            ]
          }
      }
      ...
  ]
```


#### Load AI Harm Questionnaire

**Note:** The cell below loads examples of ai harm questionnaire from Risk Atlas Master. To load your custom questionnaire, create it according to the specified format and load it instead.


In [7]:
ai_harm_questionnaire = load_resource("cset_ai_harm_questionnaire.json")

ai_harm_questionnaire[0]

{'no': 'Q1',
 'question': "Did the incident occur in a domain with physical objects? 'Yes' if the AI system(s) is embedded in hardware that can interact, affect, and change with the physical objects (roads, factories, medical facilities, etc.). 'No' if the system cannot. This includes systems that inform, detect, predict, or recommend.",
 'cot_examples': [{'intent': "Google's Perspective API, which assigns a toxicity score to online text, seems to award higher toxicity scores to content involving non-white, male, Christian, heterosexual phrases.",
   'answer': 'No'},
  {'intent': 'An Uber autonomous vehicle (AV) in autonomous mode struck and killed a pedestrian in Tempe, Arizona.',
   'answer': 'Yes'}],
 'response_format': {'type': 'object',
  'properties': {'answer': {'type': 'string', 'enum': ['Yes', 'No', 'Maybe']},
   'explanation': {'type': 'string'}},
  'required': ['answer', 'explanation']}}

There are two ways to use the inference engine to get the LLM outputs. `generate_zero_shot_risk_questionnaire_output` which gives the zero-shot output for the question and `generate_few_shot_risk_questionnaire_output` which gives the output using few-shot examples defined above.


In [8]:
usecase = "Using AI to  write a government policy report on automated decision-making in Australia's welfare system. The task will include  drafting a comprehensive section analyzing input documents to understand how Australian federal courts have addressed automated penalties in welfare systems and ensure the output is factually correct."

results = risk_atlas_nexus.generate_few_shot_risk_questionnaire_output(
    usecase,
    ai_harm_questionnaire,
    inference_engine,
)

for index, (risk_question, result) in enumerate(
    zip(ai_harm_questionnaire, results), start=1
):
    print(f"\nQ{index}: " + risk_question["question"] + "\nA: " + str(result.prediction))

Inferring with RITS: 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


Q1: Did the incident occur in a domain with physical objects? 'Yes' if the AI system(s) is embedded in hardware that can interact, affect, and change with the physical objects (roads, factories, medical facilities, etc.). 'No' if the system cannot. This includes systems that inform, detect, predict, or recommend.
A: {'answer': 'No', 'explanation': "The task described involves using AI to draft a government policy report, which is a domain of information and analysis rather than direct interaction with physical objects. The AI system is being used to analyze input documents and understand court rulings on automated penalties in Australia's welfare system. This process does not involve physical interaction with objects like roads, factories, or medical facilities. Instead, it focuses on processing and interpreting data from legal documents."}

Q2: Did the AI incident occur in the entertainment industry? 'Yes' if the sector in which the AI was used is associated with entertainment. 'No' 